# Modeling complex time series

![Sample Image](/Users/maukanmir/Documents/Machine-Learning/AI-ML-Textbooks/AI-ML-Learning/images/ARMA-process.png)


## Examining the autoregressive moving average process
#### The autoregressive moving average process is a combination of the autoregressive process and the moving average process. It states that the present value is linearly dependent on its own previous values and a constant, just like in an autoregressive process, as well as on the mean of the series, the current error term, and past error terms, like in a moving average process

#### The autoregressive moving average process is denoted as ARMA(p,q), where p is the order of the autoregressive portion, and q is the order of the moving average portion. Mathematically, the ARMA(p,q) process is expressed as a linear combination of a constant C, the past values of the series yt–p, the mean of the series µ, past error terms ϵt–q, and the current error term ϵt, as shown in equation 6.1.

- yt = C + ϕ1yt–1 + ϕ2yt–2 +⋅⋅⋅+ ϕpyt–p + ϵt + θ1ϵt–1 + θ2ϵt–2 +⋅⋅⋅+ θqϵt–q